## Calculate centroid of inceptionv3 code for each categories



In [ ]:
import os
import sys
import json

import sys
from processor import preprocess
from processor import DataSet


from keras.preprocessing.image import ImageDataGenerator
import numpy as np


import pandas as pd
import glob

This notebook assume data is already exist. The setup code is in train_178categories.ipynb.

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

MODEL_NAME = ""
CATEGORY_DICT_CSV=os.path.join(BASE_DATA_DIR, "")

#Data dirs {train, validation}
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))


In [ ]:
from category import load_category_dict
catdict = load_category_dict(CATEGORY_DICT_CSV)

### Model is stored at trained_model_latest using prediction_with_trained_model.ipynb

In [ ]:
MODEL_NAME = ""

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers

basemodel_layer_num = 311 #corresponding to len(base_model.layers)

def complile_extractor_model(model_prefix):
    '''    
    outpu : compiled model
    '''
    with open("{}.json".format(model_prefix), 'r') as f:
        model = model_from_json(f.read())
    model.load_weights("{}.h5".format(model_prefix))
    return Model(inputs=model.input, outputs=model.layers[312].output)

In [ ]:
extractor = complile_extractor_model(MODEL_NAME)

In [ ]:
from processor import DataSet

In [ ]:
def calc_centroid(files, extractor):
    ds = DataSet()
    datas = ds.files_to_dataset(files)
    predictions = extractor.predict(datas)
    return predictions.mean(axis=0)


In [ ]:
def collect_all_category_centroids(basedir, extractor):
    print("Collect category centroids at {}".format(basedir))
    cats = os.listdir(basedir)
    resdict = {}
    cnt = 0
    for catnum in cats:
        print("Cat: {} ({})".format(catnum, cnt))
        cnt+=1
        files = glob.glob("{}/{}/**/*.jpg".format(basedir, catnum), recursive=True)
        resdict[catnum] = calc_centroid(files, extractor)
    print("DONE")
    return resdict

In [ ]:
cents = collect_all_category_centroids(TRAIN_DATA_DIR, extractor)

In [ ]:
df = pd.DataFrame(cents)

In [ ]:
df

In [ ]:
df.to_pickle("trained_model/centroids.dat")

## From saved centroids

In [ ]:
df = pd.read_pickle("trained_model/centroids.dat")

In [ ]:
df.T.shape

## Visualize

In [ ]:
transp = df.T

In [ ]:
from sklearn import manifold

In [ ]:
tsne = manifold.TSNE(n_components=2, init='pca')

In [ ]:
categories = transp.index

In [ ]:
res = tsne.fit_transform(df.T)

In [ ]:
res

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(res[:, 0], res[:, 1])
plt.show()

In [ ]:
catdict[0]

In [ ]:
len(categories)

In [ ]:
plt.scatter(res[:, 0], res[:, 1])
for i in range(len(categories)):
    plt.text(res[i, 0], res[i, 1], catdict[int(categories[i])])
plt.show()

In [ ]:
plt.figure(figsize = (12,12))
plt.scatter(res[:, 0], res[:, 1])
for i in range(len(categories)):
    plt.text(res[i, 0], res[i, 1],categories[i])
plt.show()

In [ ]:
catdict[65]

In [ ]:
catdict[81]

In [ ]:
catdict[82]

In [ ]:
catdict[71]

In [ ]:
catdict[90]

In [ ]:
twopos = res[(65, 81), :]

In [ ]:
twopos

In [ ]:
plt.figure(figsize = (12,12))

labels = ["hamb", "roco"]
plt.scatter(twopos[:, 0], twopos[:, 1])
#plt.scatter(res[:, 0], res[:, 1])
for i in range(2):
    plt.text(twopos[i, 0], twopos[i, 1], labels[i])
plt.xlim(min(res[:, 0]), max(res[:, 0]))
plt.ylim(min(res[:, 1]), max(res[:, 1]))
plt.show()

In [ ]:
len(res)

In [ ]:
BOUND = 100*100+100*100

In [ ]:
isinside = list(map(lambda pair: pair[0]**2+pair[1]**2 < BOUND, [(res[i, 0], res[i, 1]) for i in range(len(res))]))

In [ ]:
filtered = res[isinside, :]
filtered_cat = categories[isinside]

In [ ]:
len(filtered)

In [ ]:
len(filtered_cat)

In [ ]:
plt.figure(figsize = (12,12))
plt.scatter(filtered[:, 0], filtered[:, 1])
for i in range(len(filtered_cat)):
    plt.text(filtered[i, 0], filtered[i, 1],filtered_cat[i])
plt.show()

In [ ]:
(catdict[24], catdict[136])

In [ ]:
(catdict[9], catdict[67])

In [ ]:
(catdict[25], catdict[48])

In [ ]:
(catdict[127], catdict[132])

In [ ]:
(catdict[47], catdict[73])

In [ ]:
(catdict[76], catdict[124])

In [ ]:
(catdict[16], catdict[32])

In [ ]:
(catdict[6], catdict[106])

In [ ]:
(catdict[137], catdict[170])

In [ ]:
178*178

## Calc L2 norm matrix

In [ ]:
transp[0:5]

In [ ]:
len(transp)

In [ ]:
vect = transp.iloc[0]

In [ ]:
((transp - vect)**2).sum(axis=1)

In [ ]:
transp.index[3]

In [ ]:
dist2dic = {}

for cat in range(len(transp)):
    catname = transp.index[cat]
    vect = transp.iloc[cat]
    dist2dic[catname] = ((transp - vect)**2).sum(axis=1)

In [ ]:
dist2df = pd.DataFrame(dist2dic)

In [ ]:
dist2df

In [ ]:
catdictst = {str(key):val for key, val in catdict.items()}

In [ ]:
top3 = dist2df.nsmallest(3, "0")
print(catdictst[top3.index[0]], catdictst[top3.index[1]], catdictst[top3.index[2]])
top3

In [ ]:
top3 = dist2df.nsmallest(3, "1")
print(catdictst[top3.index[0]], catdictst[top3.index[1]], catdictst[top3.index[2]])
top3

In [ ]:
top3

In [ ]:
top3["1"][1]

In [ ]:
def check_top3(catname):
    top3 = dist2df.nsmallest(3, catname)
    return (catdictst[top3.index[0]], "{}:{:.2f}".format(catdictst[top3.index[1]], top3[catname][1]), "{}:{:.2f}".format(catdictst[top3.index[2]], top3[catname][2]))

In [ ]:
check_top3("0")

In [ ]:
for i in range(178):
    print(check_top3(str(i)))

In [ ]:
"0" in handled

In [ ]:
handled.add("0")

In [ ]:
"0" in handled

In [ ]:
len(dist2df)

In [ ]:
[1,2] + [3, 4]

In [ ]:
from category import merge_categories_below
marge_categories = marge_categories_below

In [ ]:
catlist60 = merge_categories(dist2df, 60.0)

In [ ]:
def catlist2names(catlist, catdictst):
    return ["_".join(cats2) for cats2 in 
               [map(lambda cat: catdictst[cat], cats) for cats in catlist]]

In [ ]:
catlist60[0:3]

In [ ]:
len(catlist60)

In [ ]:
catlist2names(catlist60, catdictst)

In [ ]:
catlist80 = merge_categories(dist2df, 80.0)

In [ ]:
len(catlist80)

In [ ]:
catlist80[0:5]

In [ ]:
dic = {}

In [ ]:
catlist2names(catlist80, catdictst)

In [ ]:
catlist100 = merge_categories(dist2df, 100.0)

In [ ]:
len(catlist100)

In [ ]:
catlist2names(catlist100, catdictst)

In [ ]:
catlist120 = merge_categories(dist2df, 120.0)

In [ ]:
len(catlist120)

In [ ]:
catlist2names(catlist120, catdictst)

In [ ]:
len(merge_categories(dist2df, 150.0))

In [ ]:
catlist150 = merge_categories(dist2df, 150.0)

In [ ]:
catlist2names(catlist150, catdictst)

In [ ]:
handled = set()
thrshold = 60.0
catlist = []

In [ ]:
for cat in dist2df:
    if cat not in handled:
        samecats = dist2df.index[dist2df[cat] < thrshold]
        catlist.append(samecats)
        list(map(lambda x: handled.add(x), samecats))


In [ ]:
catlist

In [ ]:
catlist

In [ ]:
"_".join(catlist[1])

In [ ]:
len(catlist)

In [ ]:
catdictst[catlist[0][0]]

In [ ]:
dist2df.index[dist2df["1"] < thrshold]

In [ ]:
    top3 = dist2df.nsmallest(3, catname)


### Try and error

In [ ]:
MODEL = complile_extractor_model(MODEL_NAME)

In [ ]:
len(MODEL.layers)

In [ ]:
MODEL.layers[311:314]

In [ ]:
MODEL.layers[312]

In [ ]:
model = MODEL

In [ ]:
extractor = Model(inputs=model.input, outputs=model.layers[312].output)

In [ ]:
dataset = DataSet()

In [ ]:
hamburgfs = glob.glob("{}/65/**/*.jpg".format(TRAIN_DATA_DIR), recursive=True)

In [ ]:
len(hamburgfs)

In [ ]:
datas = dataset.files_to_dataset(hamburgfs)

In [ ]:
predictions = extractor.predict(datas)

In [ ]:
predictions.shape

In [ ]:
np.mean(predictions)

In [ ]:
predictions.mean(axis=0).shape

In [ ]:
predictions.mean(axis=0)

In [ ]:
hamburgcentro = predictions.mean(axis=0)

In [ ]:
actual = calc_centroid(glob.glob("{}/65/**/*.jpg".format(TRAIN_DATA_DIR), recursive=True), extractor)

In [ ]:
all(hamburgcentro == actual)

In [ ]:
categories = os.listdir(TRAIN_DATA_DIR)

In [ ]:
ds = DataSet()

In [ ]:
res = calc_centroid(glob.glob("{}/{}/**/*.jpg".format(TRAIN_DATA_DIR, "50"), recursive=True), extractor)

In [ ]:
len(ds.deb)

In [ ]:
shp = ds.deb[0].shape

In [ ]:
shp

In [ ]:
[arr.shape for arr in ds.deb if arr.shape != shp]

In [ ]:
shp == ds.deb[1].shape

In [ ]:
shps = [arr.shape for arr in ds.deb]

In [ ]:
shps[0:5]

In [ ]:
shps.index((224, 224, 4))

In [ ]:
glob.glob("{}/{}/**/*.jpg".format(TRAIN_DATA_DIR, "50") , recursive=True)[221]

In [ ]:
from scipy.misc import imresize, imread

In [ ]:
target_normal = ''

In [ ]:
arr1 = imread(target_normal, mode="RGB")

In [ ]:
arr2 = imread(target_normal)

In [ ]:
(arr1 == arr2).all()

In [ ]:
target = ''

In [ ]:
arr = imread(target, mode="RGB")

In [ ]:
arr.shape

In [ ]:
help(imread)

In [ ]:
arr.shape

In [ ]:
from IPython.display import Image, display

# display(Image('))

In [ ]:
arr[0:5, 0:5, 0]

In [ ]:
del sys.modules['processor']

In [ ]:
from processor import DataSet

In [ ]:
cents.keys()[0:5]

In [ ]:
cents["65"]

In [ ]:
list(cents.items())[0:3]